# ⭐ 图像亮点识别分类模型
本项目根据图像中白点的亮度、形状特征对图像进行二分类（0 或 1）。

In [ ]:
# 📦 导入库
import cv2
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


## 🧹 图像预处理函数

In [ ]:
def preprocess_image(img_path):
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (256, 256))
    _, binary = cv2.threshold(img, 30, 255, cv2.THRESH_BINARY)
    return binary, img

## 📐 特征提取函数

In [ ]:
def extract_features(binary_img, gray_img):
    contours, _ = cv2.findContours(binary_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    num_objects = len(contours)
    avg_brightness = np.mean(gray_img[gray_img > 30]) if np.any(gray_img > 30) else 0
    avg_area = np.mean([cv2.contourArea(c) for c in contours]) if contours else 0
    avg_circularity = np.mean([
        (4 * np.pi * cv2.contourArea(c)) / (cv2.arcLength(c, True)**2 + 1e-6)
        for c in contours if cv2.arcLength(c, True) > 0
    ]) if contours else 0
    return [num_objects, avg_brightness, avg_area, avg_circularity]

## 🏗️ 构建训练数据并训练模型

In [ ]:
# 替换为你自己的图像路径
image_paths = ['NCI6_XY10000.png', 'CAR T-Cell tracks_NCI2_T001_XY1.png']
labels = [0, 1]

X, y = [], []
for path, label in zip(image_paths, labels):
    binary, gray = preprocess_image(path)
    features = extract_features(binary, gray)
    X.append(features)
    y.append(label)

clf = RandomForestClassifier(n_estimators=50)
clf.fit(X, y)
print("模型训练完成 ✅")

## 🔍 推理函数

In [ ]:
def predict_image(path, model):
    binary, gray = preprocess_image(path)
    feat = extract_features(binary, gray)
    return model.predict([feat])[0]

In [ ]:
# 预测
print("图像1 预测结果:", predict_image('NCI6_XY10000.png', clf))  # 期望 0
print("图像2 预测结果:", predict_image('CAR T-Cell tracks_NCI2_T001_XY1.png', clf))  # 期望 1